In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importing the proteins data
capsid = []
capsidd = []
        
f = open("../../../../../../Data/Proteins/DENV4/Capsid/DENV4_capsid.txt", "r")
for x in f:
    if "DSS" in x:
        capsidd.append(1)
    elif "DHF" in x:
        capsidd.append(1)
    elif x[0] == ">":
        capsidd.append(0)
    else:
        x = x.replace("\n", "")
        capsid.append(x)

In [3]:
# Converting the arrays into DataFrames
capsid = pd.DataFrame(capsid)

# Attaching the "Disease" label column to the input
capsid["Disease"] = capsidd

# Renaming the columns
capsid = capsid.rename(index=str, columns={0: "Sequence", "Disease": "Disease"})

In [4]:
# Concatinating the input data into one DataFrame
print("The combined shape of the given data is:", str(capsid.shape))
print("The length of the combined data is:", str(len(capsid.index)))

# Shuffling the data and then taking a peek
capsid = capsid.sample(frac = 1)
print(capsid.head())

The combined shape of the given data is: (50, 2)
The length of the combined data is: 50
                                             Sequence  Disease
36  MNQRKKVVRPPFNMLKRERNRVSTPQGLVKRFSTGLFSGKGPLRMV...        0
10  MNQRKKVVRPPFNMLKRERNRVSTPQGLVKRFSTGLFSGKGPLRMV...        0
2   MNQRKKVVRPPFNMLKRERNRVSTPQGLVKRFSTGLFSGKGPLRMV...        0
6   MNQRKKVVRPPFNMLKRERNRVSTPQGLVKRFSTGLFSGKGPLRMV...        0
45  MNQRKKVVRPPFNMLKRERNRVSTPQGLVKRFSTGLFSGKGPLRMV...        1


In [5]:
# Function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size = 6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

capsid['words'] = capsid.apply(lambda x: getKmers(x['Sequence']), axis=1)
capsid = capsid.drop('Sequence', axis=1)

capsid_texts = list(capsid['words'])
for item in range(len(capsid_texts)):
    capsid_texts[item] = ' '.join(capsid_texts[item])

In [6]:
# Creating y and printing the shape of it
y = capsid.iloc[:, 0].values
print("The shape of y is:", y.shape)

The shape of y is: (50,)


In [7]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
x = cv.fit_transform(capsid_texts)

# Print the shape of x
print("The shape of x matrix is:", x.shape)

The shape of x matrix is: (50, 207)


## Nested Cross-Validation

In [8]:
from sklearn.model_selection import cross_val_score
# search for an optimal value of C for Logistic Regression
Tree_list = [10, 100, 1000, 2500, 5000]
# empty list to store scores
RF_scores = []

In [9]:
from sklearn.ensemble import RandomForestClassifier
# 1. we will loop through reasonable values of k
for trees in Tree_list:
    # 2. run RandomForestClassifier with trees trees
    lr = RandomForestClassifier(n_estimators = trees, criterion = 'entropy', random_state = 0)
    # 3. obtain cross_val_score for RandomForestClassifier with trees trees
    scores = cross_val_score(lr, x, y, cv = 10, scoring='accuracy')
    # 4. append mean of scores for trees trees to RF_scores list
    RF_scores.append(scores.mean())
    
    print("The score for", trees, "trees, is:", scores.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for 10 trees, is: 0.85
The score for 100 trees, is: 0.85


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for 1000 trees, is: 0.85


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for 2500 trees, is: 0.85


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for 5000 trees, is: 0.85
